In [ ]:
import torch
import leap
from leap import LeapForCausalLM, LeapConfig
from lstm import LstmForCausalLM
from transformers import (PreTrainedTokenizerFast, TrainingArguments,
                          Trainer, default_data_collator,
                          GPT2Config, GPT2LMHeadModel)

from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from torch.utils.data import Subset

# word level tokenizer as per wikitext modeling
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer

import math
import copy
from itertools import chain

import logging
logging.disable(logging.INFO)

In [ ]:
# globals
raw_datasets = load_dataset("wikitext", "wikitext-103-v1")
total_train_tokens = 105268829 # see appendix at the end of notebook
max_num_params = 69308416
param_data_ratio = max_num_params / total_train_tokens
seq_len = 1024
subset_datasets = raw_datasets

# hyperparameters
training_args = TrainingArguments(
    output_dir = "./results",
    logging_strategy = "steps",
    evaluation_strategy = "steps",
    logging_steps = 2000,
    report_to = "none",
    learning_rate = 5e-4,
    lr_scheduler_type = "cosine",
    warmup_ratio = .05,
    num_train_epochs = 1,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    max_grad_norm = 1,
    fp16 = True,
)

Found cached dataset wikitext (C:/Users/micha/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

# PREPROCESSING

In [ ]:
# make a word level tokenizer
tokenizer = Tokenizer(WordLevel(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
tokenizer.enable_padding(pad_id = 0, pad_token = "<pad>")
# no post processing

# only use vocab size of 8192 for reasonable speed/memory
token_trainer = WordLevelTrainer(vocab_size = 8191, # -1 for pad token
                                 special_tokens = ["<unk>"])

def batch_iterator(batch_size=10000):
    text = raw_datasets["train"]['text']
    for i in range(0, len(text), batch_size):
        yield text[i : i + batch_size]

tokenizer.train_from_iterator(batch_iterator(),
                              trainer = token_trainer,
                              length = len(raw_datasets["train"]["text"]))
tokenizer = PreTrainedTokenizerFast(tokenizer_object = tokenizer, pad_token = "<pad>")

# tokenized the dataset
def tokenize_function(examples):
    output = tokenizer(examples["text"])
    return output

# tokenize dataset
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched = True,
    remove_columns = "text",
    desc = f"tokenize dataset",
    load_from_cache_file = True
)

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    # Split by chunks of max_len
    result = {
        k: [t[i : i + seq_len] for i in range(0, total_length, seq_len)]
        for k, t in concatenated_examples.items()
    }
    
    # for language modeling, inputs are labels (they will be shifted inside the model)
    result["labels"] = result["input_ids"].copy()
    
    # pad last block with 0
    last_ids = result["input_ids"][-1]
    diff = seq_len - len(last_ids)
    result["input_ids"][-1] = last_ids + [0 for _ in range(diff)]
    
    # set attention mask to mask out these tokens
    result["attention_mask"][-1] = result["attention_mask"][-1] + [0 for _ in range(diff)]
    
    # set pad labels to -100 so they will be ignored by CrossEntropyLoss
    result["labels"][-1] = result["labels"][-1] + [-100 for _ in range(diff)]
    return result

lm_dataset = tokenized_datasets.map(
    group_texts,
    batched = True,
    batch_size = 10000,
    desc = f"Grouping texts in chunks of {seq_len}",
    load_from_cache_file = True
)

lm_dataset = lm_dataset.remove_columns(["token_type_ids"])

Loading cached processed dataset at C:/Users/micha/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-a2031eb206d20f87.arrow
Loading cached processed dataset at C:/Users/micha/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-8f3dd2e5d5819fe2.arrow
Loading cached processed dataset at C:/Users/micha/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-f4668efef485cbea.arrow
Loading cached processed dataset at C:/Users/micha/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-9004e9678f0a1614.arrow
Loading cached processed dataset at C:/Users/micha/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-d18

## helper function

In [ ]:
def subset_data(dataset, num_parameters, param_data_ratio):
    dataset = DatasetDict(copy.deepcopy(dataset))
    subset_num_tokens = num_parameters / param_data_ratio
    
    global seq_len
    num_rows = int(subset_num_tokens) // seq_len

    training_set = dataset["train"]
    dataset["train"] = Dataset.from_dict(training_set[:num_rows+1])
    
    real_num_tokens = len(dataset["train"]) * seq_len
    print(f'NUMBER OF TOKENS: {real_num_tokens}, with commas {real_num_tokens:,}')
    
    dataset.set_format('pt')
    return dataset

# TRAINING FUNCTION

In [ ]:
def run_training(hidden_size, n_head = None):
    # calculate number of layers needed based on levine 2020
    n_layer = round((math.log(hidden_size) - 5.039) / 5.55e-2)
    n_layer = max(1, n_layer)
    print(f'Using {n_layer} layers')
    
    config = LeapConfig(
        hidden_size = hidden_size, n_layer = n_layer, n_head = n_head,
        vocab_size = len(tokenizer) + 1, n_positions = seq_len,
        use_local_att = True, window_sizes = None, rescale = 10,
        initializer_range = 1 / hidden_size**.5, hidden_dropout_prob = 0 # no dropout bc one epoch
    )
    model = LeapForCausalLM(config)

    # get number of parameters
    total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # get rid of embedding parameters
    token_embeddings = (len(tokenizer) + 1) * hidden_size
    position_embeddings = seq_len * hidden_size
    non_embedding_parameters = total_parameters - token_embeddings - position_embeddings
    print(f'NON EMBEDDING PARAMETERS: {non_embedding_parameters}, with commas {non_embedding_parameters:,}')

    # subset dataset using global lm_dataset
    global lm_dataset
    subset_datasets = subset_data(lm_dataset, non_embedding_parameters, param_data_ratio)

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=default_data_collator,
        train_dataset=subset_datasets["train"],
        eval_dataset=subset_datasets["validation"],
    )

    trainer.train()
    
    print("\n===============VALIDATION SET CROSS ENTROPY LOSS EVALUATION===============\n")
    print(trainer.evaluate(subset_datasets["validation"]))

    # save gpu memory
    del trainer
    del model
    del subset_datasets
    torch.cuda.empty_cache()

# Ablations

The idea here is instead of changing the source code in LEAP.py over and over again and then running personal experiments, that it makes much more sense to record this stuff in a notebook. Still it seems like a bad option to have to write a bunch of if statements about which little abalation to use. The solution, monkey patching! We will rewrite the LEAP forward function before each abalation test. An example is shown, and this notebook will be annotated

## original forward function

In [ ]:
def forward(self, q, f, k, v, attention_mask = None):        
        batch_size, seq_len, hidden_size = v.shape
        
        # reshape for multihead formulation
        q = q.reshape(batch_size, seq_len, self.n_head, self.head_size)
        f = f.reshape(batch_size, seq_len, self.n_head, self.head_size)
        k = k.reshape(batch_size, seq_len, self.n_head, self.head_size)
        v = v.reshape(batch_size, seq_len, self.n_head, self.head_size)
        
        # unparameterized norming of vectors so dot products don't explode (also why it is after reshaping)
        if self.rescale:
            q = self.__real_norm(q)
            f = self.__real_norm(f)
            k = self.__real_norm(k)
            v = self.__real_norm(v)
            
        # dropout regularization (keys don't need dropout as they are always dotted with a dropped out vector)
        q = self.drop(q)
        f = self.drop(f)
        v = self.drop(v)

        # manual "matrix dot product" for speed (in einsum notation "bshe, bshe->bsh") with scaling
        focus_logits = (f * k).sum(dim = -1) * self.scaling_factor
        
        # apply dropout to logits so that all tokens will have a chance at getting focus
        focus_logits = self.drop(focus_logits)
        
        # masking out pad tokens
        if attention_mask is not None:
            focus_logits += attention_mask
        
        # manual softmax within cumulative sum
        focus_weights = torch.exp(focus_logits)
        focus_weights = focus_weights.unsqueeze(-1)
        
        # normalization term for softmax
        cumulative_weights = torch.cumsum(focus_weights, dim = 1)
        cumulative_weights = cumulative_weights - self.__window_align(cumulative_weights)
        
        focused_k = self.__w_focus(focus_weights, cumulative_weights, k)
        focused_v = self.__w_focus(focus_weights, cumulative_weights, v)
        
        # querying by measuring dot product alignment (with scaling)
        alignment = torch.sigmoid((q * focused_k).sum(dim = -1) * self.scaling_factor)
        attention = alignment.unsqueeze(-1) * focused_v
        
        # concat heads
        attention = focused_v.reshape(batch_size, seq_len, hidden_size)
        
        return attention

## monkey patched

In [ ]:
def forward(self, q, f, k, v, attention_mask = None):        
    print("hi, we abalate the entirety of LEAP here :)", end = '\r')
    return v

leap.MultiheadLeap.forward = forward

In [ ]:
run_training(64, n_head = 1)

Using 1 layers
NON EMBEDDING PARAMETERS: 49856, with commas 49,856
NUMBER OF TOKENS: 75776, with commas 75,776


Step,Training Loss,Validation Loss


hi, we abalate the entirety of LEAP here :)
===============VALIDATION SET CROSS ENTROPY LOSS EVALUATION===============



{'eval_loss': 8.707731246948242, 'eval_runtime': 0.553, 'eval_samples_per_second': 383.364, 'eval_steps_per_second': 191.682, 'epoch': 1.0}
